<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%BA%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup
import folium # 新增 folium 導入
from io import StringIO # 新增 StringIO 導入

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('int')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')
SHEET_URL = "https://docs.google.com/spreadsheets/d/1myM3mFwjKdn0aJdENwhue18m-gwCDFLoPKpMqw6t1L0/edit?usp=sharing"
WORKSHEET_NAME = "模擬資料"
TIMEZONE = "Asia/Taipei"

# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df_restaurants = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df_restaurants.head()

,餐廳名稱,價位,距離,是否營業,特色描述
0,雙月食品社,$,短,是,米其林雞湯/中式
1,溫州街蘿蔔絲餅,$,短,是,經典點心/便宜
2,杭州小籠湯包,$$,中,是,平價小籠包
3,瑞安豆漿大王,$,短,否,24H中式早餐
4,不二煮藝,$$,中,是,平價義式/健康


In [ ]:
# -----------------------------------------------
# 1. 地圖資料載入 (使用 df_map_data)
# -----------------------------------------------

# 全域變數：地圖中心點 (假設在附近)
MAP_CENTER_LAT = 25.029800
MAP_CENTER_LNG = 121.531500

# *** 模擬地圖所需的經緯度資料 ***
# 這部分是您提供的模擬地圖資料，現在確保它在 Gradio 介面定義前
map_lookup = {
    '雙月食品社': (25.027472, 121.529938),
    '溫州街蘿蔔絲餅': (25.027800, 121.530500),
    '杭州小籠湯包': (25.031500, 121.535000),
    '瑞安豆漿大王': (25.032200, 121.532100),
    '不二煮藝': (25.031000, 121.531500),
    '麵屋黑心': (25.028500, 121.530100),
    '樂業麵線': (25.030000, 121.533000),
    '青島水餃': (25.029000, 121.531000),
    '永和豆漿': (25.030800, 121.534000)
}
# 假設我們將這個查找表轉換成一個 DataFrame 以便後續操作
df_map_data = pd.DataFrame(
    list(map_lookup.items()),
    columns=['餐廳名稱', 'Coords']
)
df_map_data[['lat', 'lng']] = pd.DataFrame(df_map_data['Coords'].tolist(), index=df_map_data.index)
df_map_data = df_map_data.drop(columns=['Coords'])
# -----------------------------------------------

In [ ]:
# -----------------------------------------------
# 2. 核心功能函數 (修正 generate_folium_map)
# -----------------------------------------------

def generate_folium_map(selected_restaurant_name, df_map_data_input): # <--- 修正: 加上 df_map_data_input
    """根據選定的餐廳名稱，生成 Folium 地圖 HTML"""

    if not selected_restaurant_name:
        # 如果沒有選擇餐廳，返回一個空的或中心地圖
        m = folium.Map(location=[MAP_CENTER_LAT, MAP_CENTER_LNG], zoom_start=15)
        return m._repr_html_()

    # 查找選定餐廳的經緯度 (使用傳入的參數)
    match = df_map_data_input[df_map_data_input['餐廳名稱'] == selected_restaurant_name]

    if match.empty:
        # 如果找不到，返回中心地圖
        m = folium.Map(location=[MAP_CENTER_LAT, MAP_CENTER_LNG], zoom_start=15)
        return m._repr_html_()

    # 取得經緯度
    lat = float(match['lat'].iloc[0])
    lng = float(match['lng'].iloc[0])

    # 創建地圖，以餐廳為中心
    m = folium.Map(location=[lat, lng], zoom_start=17)

    # 添加標記
    folium.Marker(
        location=[lat, lng],
        tooltip=selected_restaurant_name,
        popup=f"**{selected_restaurant_name}** - 推薦地點",
        icon=folium.Icon(icon="cutlery", prefix='fa', color='blue'),
    ).add_to(m)

    # 返回 HTML
    return m._repr_html_()

# --- filter_and_recommend (保持不變) ---
def filter_and_recommend(df, price_filter, distance_filter, is_available):
    # ... (函數內容保持不變) ...
    """根據使用者輸入篩選並隨機推薦 3 家餐廳"""

    # 轉換篩選條件為列表
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    # Gradio CheckboxGroup 回傳列表，但單一勾選時需處理
    available_status = ['是'] if is_available else ['是', '否']

    # 應用篩選條件
    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    # 隨機抽取 3 家，不足 3 家則全數取出
    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    # 準備推薦清單文字
    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    # 準備 AI 摘要的輸入字串
    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input

# --- generate_summary (保持不變) ---
async def generate_summary(ai_input):
    # ... (函數內容保持不變) ...
    """呼叫 LLM 產生決策建議摘要 (使用 gemini-2.5-pro)"""
    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"

    # Fallback if model configuration failed
    await asyncio.sleep(1) # 模擬網路延遲
    mock_summary = f"""
    **AI 助理 (模擬):** 今天的選擇是：{'、'.join([r.split('(')[0].strip() for r in ai_input.split('、')])}。
    如果您追求**效率和銅板價**，{ai_input.split('、')[0].split('，')[0]} (價位{ai_input.split('、')[0].split('，')[1].split('價位')[1]}) 肯定是首選，快速又飽足。
    若想來點**儀式感和高品質**，{'、'.join(ai_input.split('、')[1:])} 提供了更好的用餐體驗，但距離和花費會稍高一點。
    決定權在你！
    (請確保您已安裝 'google-genai' 並在 Colab Secrets 中設置 'gemini' API key 以啟用真實 AI 推薦)
    """
    return mock_summary.strip()


    system_prompt = (
        "您是一位幽默且極具效率的午餐決策顧問。您的任務是根據提供的餐廳清單，"
        "分析其優缺點（如價格、距離、特色）並總結，幫助使用者在 3 家中快速做出決定。"
        "請以輕鬆口吻，用 **繁體中文** 撰寫一個約 **100 字** 的決策建議摘要，不超過 150 字。"
        "請避免使用 Markdown 標題和編號清單。"
    )

    user_query = f"這是三家推薦的餐廳清單：{ai_input}。請綜合考量其價格、距離和特色，為我提供一份快速決策建議。"

    try:
        # 使用 asyncio.to_thread 讓同步的 SDK 呼叫不會阻塞 Gradio 的事件迴圈
        response = await asyncio.to_thread(
            model.generate_content,
            contents=user_query,
            config=genai.types.GenerateContentConfig(
                system_instruction=system_prompt,
                temperature=0.7,
            )
        )
        return response.text.strip()

    except Exception as e:
        error_msg = f"呼叫 Gemini API 失敗，請檢查 API 金鑰或配額：{e}。"
        return error_msg

In [ ]:
# -----------------------------------------------
# 3. 介面更新函數 (修正: wrapper_recommend_with_json)
# -----------------------------------------------

async def wrapper_recommend_with_json(price_filter, distance_filter, is_available):
    """修正: Gradio 主要流程：篩選 -> 推薦 -> 呼叫 AI 摘要，並輸出 Dataframe JSON"""

    # 1. 篩選與推薦
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_restaurants, price_filter, distance_filter, is_available
    )

    # 2. 呼叫 AI 摘要
    ai_summary = await generate_summary(ai_input)

    # 3. 準備給 Gradio 輸出
    # 將推薦結果轉換為 Dataframe (僅顯示名稱、價位、距離)
    display_df = recommended_df[['餐廳名稱', '價位', '距離']].reset_index(drop=True)

    # 4. 將完整推薦結果轉為 JSON 字串，用於 State 傳輸給地圖函式
    recommended_df_json_output = recommended_df.to_json()

    # 處理空結果
    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", "（無建議）", "" # <--- 修正回傳值數量

    return display_df, recommend_list, ai_summary, recommended_df_json_output # <--- 修正回傳值數量

# -----------------------------------------------
# 4. 點擊事件處理函數 (修正: handle_row_selection)
# -----------------------------------------------
# 提取被點擊行資料的函數 (Gradio DataFrame 的點擊事件回傳一個事件物件)
def handle_row_selection(evt: gr.SelectData, recommended_df_json, df_map_data_input): # <--- 修正: 加上 df_map_data_input
    """處理 DataFrame 行選擇事件，提取餐廳名稱並呼叫地圖生成函數"""
    # 將 JSON 字串轉換回 DataFrame
    if not recommended_df_json:
        return generate_folium_map(None, df_map_data_input) # <--- 修正: 傳入 df_map_data_input

    # 使用 StringIO 處理 JSON 字串，以確保與 Gradio 的傳輸兼容性
    recommended_df = pd.read_json(StringIO(recommended_df_json))

    # 檢查事件是否包含行索引 (evt.index[0] 是行索引)
    if evt.index and len(evt.index) > 0:
        row_index = evt.index[0]
        selected_restaurant_name = recommended_df.iloc[row_index]['餐廳名稱']
        return generate_folium_map(selected_restaurant_name, df_map_data_input) # <--- 修正: 傳入 df_map_data_input
    else:
        return generate_folium_map(None, df_map_data_input) # <--- 修正: 傳入 df_map_data_input

In [ ]:
# -----------------------------------------------
# 5. Gradio 介面配置 (保持不變，因為綁定已經是正確的)
# -----------------------------------------------

# 設定 Gradio 介面元件
with gr.Blocks(title="迷你黑客松午餐決策器") as demo:
    gr.Markdown(
        """
        # 🍽️ 迷你黑客松午餐/聚會決策器
        ### 目標：從餐廳清單中篩選，並隨機推薦 3 家，最後由 AI 助您快速決策！
        **🚨 注意：請確保您已在 Colab 中讀取 Google Sheet 數據，否則將使用模擬數據。**
        """
    )

    # 1. 新增一個 State 元件，用於在推薦後傳輸完整的 DataFrame 資料
    recommended_df_json = gr.State(value="")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 條件篩選")
            price_select = gr.CheckboxGroup(
                ['$', '$$', '$$$'],
                label="價位過濾 (可複選)",
                value=['$', '$$'],
                info="($: 銅板/150內, $$: 150-300, $$$: 300+)"
            )
            distance_select = gr.CheckboxGroup(
                ['短', '中', '長'],
                label="距離過濾 (可複選)",
                value=['短', '中'],
                info="(短: 步行5分, 中: 步行10分, 長: 需騎車)"
            )
            available_toggle = gr.Checkbox(
                label="只看今日營業",
                value=True
            )
            recommend_btn = gr.Button("🚀 開始推薦")

        with gr.Column(scale=2):
            gr.Markdown("## 推薦清單 (隨機 3 家)")
            # 設定 interactive=True 以啟用行選擇
            output_dataframe = gr.DataFrame(
                headers=["餐廳名稱", "價位", "距離"],
                datatype=["str", "str", "str"],
                label="隨機推薦結果",
                type="pandas",
                interactive=True,
            )
            output_list = gr.Textbox(label="推薦清單細節")

    # 2. 新增地圖顯示元件
    gr.Markdown("## 📍 餐廳位置地圖 (點擊上方列表中的餐廳)")
    map_output = gr.HTML(
        generate_folium_map(None, df_map_data), # 修正: 初始化時傳入 df_map_data
        label="互動式地圖",
        elem_id="map_container"
    )

    gr.Markdown("## 🤖 AI 決策建議 (約 100 字)")
    output_summary = gr.Textbox(
        label="AI 決策顧問報告",
        interactive=False,
        info="AI 將根據推薦結果，總結優缺點，幫助您快速做出選擇。"
    )

    # 3. 定義點擊按鈕後的行為 (綁定到修正後的 wrapper_recommend_with_json)
    recommend_btn.click(
        fn=wrapper_recommend_with_json,
        inputs=[
            price_select,
            distance_select,
            available_toggle
        ],
        # 修正: 輸出增加 recommended_df_json
        outputs=[output_dataframe, output_list, output_summary, recommended_df_json]
    )

    # 4. 綁定 DataFrame 的行選擇事件，觸發地圖更新
    output_dataframe.select(
        fn=handle_row_selection,
        # 修正: 傳入 State 元件和全域地圖資料
        inputs=[recommended_df_json, gr.State(df_map_data)],
        outputs=[map_output]
    )

    gr.Markdown(
        """
        ---
        **[開發者備註]** AI 摘要已設定為使用 `gemini-2.5-pro` 呼叫。請確保您已執行以下步驟：
        1. 在 Colab 中安裝 `google-genai` 等必要套件。
        2. 在 Colab Secrets 中設定名為 `gemini` 的 API 金鑰。
        3. **確保 `df_map_data` (包含 lat/lng) 和 `df_restaurants` (用於篩選) 兩組資料皆已載入。**
        """
    )


# 運行 Gradio 應用
if __name__ == "__main__":
    # 在 Colab 或一般環境中運行
    print("Gradio 應用正在啟動...")
    demo.launch(inbrowser=True)

Gradio 應用正在啟動...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://914f97c5b23872ce00.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
